# Python Generators

Erinnerung: mit dem `yield` Keyword wird eine Funktion automatisch zum [Generator](https://en.wikipedia.org/wiki/Generator_(computer_programming)). Ein Generator ist ein Objekt, das iteriert werden kann, also als Ausdruck nach `in` in einer `for ... in <expression>` stehen kann.

## Die Fibonacci-Folge

Wir möchten wiederverwendbaren Code schreiben, der die Fibonacci-Folge generiert.

Unser erster Versuch ist eine Funktion, die die ersten `n` Elemente der Fibonacci-Folge generiert und in einer Liste zurückgibt:

In [23]:
def fibonacci(limit=10):
    result = []
    one = 0
    two = 1
    for i in range(limit):
        one, two = two, one + two
        result.append(one)
    return result

print(fibonacci(10))

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


Das Problem ist nur, dass die Liste viel Speicher frisst, wenn wir eine grosse Anzahl Elemente benötigen. Beispiel: wir möchten die Summe der ersten `n` Fibonacci-Zahlen berechnen, ohne den Speicherplatz für die Liste aufzuwenden. Oder wir wissen gar nicht im Voraus, wieviele Elemente nötig sind, wenn wir zum Beispiel die erste Fibonacci-Zahl grösser als 1000 suchen.

Natürlich könnten wir für beide Probleme eine eigene, effiziente Funktion schreiben, die ohne Liste auskommt - aber als faule Informatiker möchten wir es vermeiden, den gleichen Code zweimal zu schreiben...

Gesucht ist also ein Stück Code, das einen _Strom_ von Fibonacci-Zahlen liefert, die wir kontinuierlich konsumieren können, bis wir genug haben.

## Iteratoren

Soche Objekte heissten _Iteratoren_. Ein Iterator hat nur eine einzige Funktion, `__next__`, die jeweils das nächste Element zurückgibt. Der Iterator muss sich irgendwie den Zustand merken, also wo in der Fibonacci-Folge er gerade steckt.

In [26]:
class FiboIterator:
    def __init__(self):
        self.one = 0
        self.two = 1
    
    def __next__(self):
        self.one, self.two = self.two, self.one + self.two
        return self.one

iterator = FiboIterator()
res = 0
count = 0
while res < 1000:
    res = next(iterator)  # Die eingebaute Funktion next ruft __next__ auf.
    count += 1
print(f'Erste Fibonacci-Zahl grösser als 1000 ist {res} (#{count})')


Erste Fibonacci-Zahl grösser als 1000 ist 1597 (#17)


# Iterables

Möchten wir unseren Iterator in einer `for`-Schleife einsetzen, müssen wir zusätzlich das Iterable-Protokoll beachten: Iterables müssen eine `__iter__` Funktion haben, die einen neuen, von vorne beginnenden Iterator zurückgibt. Wieso benötigen wir sowohl Iterable als auch Iterator? Stell dir vor, eine Liste möchte das Iterable-Protokoll implementieren - es ist entscheidend, dass jede neue Schleife wieder von vorne beginnt:

```python
alphabet = 'abcdef'
for letter1 in alphabet:      # Erster Iterator über die Sequence alphabet
    for letter2 in alphabet:  # Zweiter, unabhängiger Iterator über die gleiche Sequence
        print(letter1 + letters)
```

Spickzettel:
  * `__iter__` wird einmal aufgerufen, wenn die `for`-Schleife startet.
  * `__next__` wird vor jedem Schleifendurchgang aufgerufen und das Resultat der Schleifenvariable zugewiesen.

Die Funktionen werden [hier beschrieben](https://docs.python.org/3/library/stdtypes.html#iterator-types].

In [29]:

class Fibo:
    # Wird einmal aufgerufen, wenn Python den die for-Schleife startet.
    def __iter__(self):
        return FiboIterator()
    
sequence = Fibo()
for num in sequence:
    if num > 1000:
        print(f'Erste Fibonacci-Zahl grösser als 1000 ist {res}')
        break


Erste Fibonacci-Zahl grösser als 1000 ist 1597


Falls die Werte berechnet werden, können `__iter__` und `__next__` auch von derselben Klasse implementiert werden.

## Yield

Das ganze leuchtet ein, aber ist ziemlich kompliziert. Mit einem Generator und `yield` wird alles viel kompakter.

In [30]:
def fibo():
    one = 0
    two = 1
    while True:
        one, two = two, one + two
        yield one

for num in fibo():
    if num > 1000:
        print(f'Erste Fibonacci-Zahl grösser als 1000 ist {res}')
        break


Erste Fibonacci-Zahl grösser als 1000 ist 1597


### StopIteration

Sind keine Elemente mehr übrig, wird einfach nichts zurückgegeben bzw. ein normales `return`:


In [33]:
def fibo(limit=-1):
    one = 0
    two = 1
    while limit != 0:
        limit -= 1
        one, two = two, one + two
        yield one

for num in fibo(10):
    print(num)

n = 100
print(f'Summe der ersten {n} Fibonacci-Zahlen: {sum(fibo(n))}')


1
1
2
3
5
8
13
21
34
55
Summe der ersten 100 Fibonacci-Zahlen: 927372692193078999175
